In [ ]:
import os
import pandas as pd
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the directories and output path
directory = r'D:\SamsungSTF\Processed_Data\TripByTrip'
output_path = r'C:\Users\BSL\Desktop\result_files_acc.csv'
destination_directory = r'D:\SamsungSTF\Processed_Data\MissingData'
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)
    
# Function to check if a file contains acceleration values >= 9.8
def check_file(filepath):
    try:
        df = pd.read_csv(filepath)
        if 'acceleration' in df.columns:
            if (df['acceleration'].abs() >= 7).any():
                return os.path.basename(filepath)
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
    return None

def main():
    # Get list of all CSV files in the directory
    csv_files = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith(".csv")]

    files_with_high_acceleration = []

    # Create a ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=16) as executor:
        # Map the check_file function to the file paths
        futures = {executor.submit(check_file, filepath): filepath for filepath in csv_files}

        for future in as_completed(futures):
            result = future.result()
            if result:
                files_with_high_acceleration.append(result)

    # Sort the results
    files_with_high_acceleration.sort()

    # Save results to a CSV file
    result_df = pd.DataFrame(files_with_high_acceleration, columns=['filename'])
    result_df.to_csv(output_path, index=False)

    # Move files with high acceleration to the destination directory
    for filename in files_with_high_acceleration:
        source_path = os.path.join(directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.move(source_path, destination_path)

    return files_with_high_acceleration

if __name__ == "__main__":
    result_files = main()
    print(result_files)
